# Project 3 
## EDA

**Project Partner**: Star Li, Setfan Li

## Setup

In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style = "whitegrid", 
        color_codes = True,
        font_scale = 1.5)

In [3]:
data_state = pd.read_csv('4.18states.csv')

In [4]:
data_counties = pd.read_csv('abridged_couties.csv')

In [5]:
data_time_conf = pd.read_csv('time_series_covid19_confirmed_US.csv')

In [6]:
data_time_de = pd.read_csv('time_series_covid19_deaths_US.csv')

## Data Filtering (we only want U.S. data)

In [38]:
data_state.head(5)

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate
0,Alabama,US,2020-04-18 22:32:47,32.3182,-86.9023,4712,153,NaN,4559.0,1.0,100.492717,42538.0,620.0,3.247029,84000001,USA,907.206961,13.157895
1,Alaska,US,2020-04-18 22:32:47,61.3707,-152.4044,314,9,147.0,305.0,2.0,52.530410,9655.0,39.0,2.866242,84000002,USA,1615.226458,12.420382
2,American Samoa,US,NaN,-14.2710,-170.1320,0,0,NaN,NaN,60.0,0.000000,3.0,NaN,NaN,16,ASM,5.391708,NaN
3,Arizona,US,2020-04-18 22:32:47,33.7298,-111.4312,4724,180,539.0,4544.0,4.0,64.901548,51045.0,566.0,3.810330,84000004,USA,701.291175,11.981372
4,Arkansas,US,2020-04-18 22:32:47,34.9697,-92.3731,1744,38,703.0,1706.0,5.0,67.361213,24141.0,291.0,2.178899,84000005,USA,932.435235,16.685780


In [15]:
data_state = data_state.loc[data_state['Country_Region'] == 'US']

In [39]:
data_state = data_state.rename(columns={"Long_": "Long"})

## Data Cleaning

### 1. "4.18state"

- Divide the data into edge case (placeselike Diamond Princess that does not have lat and long) and general case

In [40]:
edge_case = data_state[data_state['Lat'].isnull()]

In [41]:
edge_case.head(10)

,Province_State,Country_Region,Last_Update,Lat,Long,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate
9,Diamond Princess,US,2020-04-18 22:32:47,NaN,NaN,49,0,0.0,49.0,88888.0,NaN,NaN,NaN,0.0,84088888,USA,NaN,NaN
13,Grand Princess,US,2020-04-18 22:32:47,NaN,NaN,103,0,0.0,103.0,99999.0,NaN,NaN,NaN,0.0,84099999,USA,NaN,NaN
116,Recovered,US,NaN,NaN,NaN,0,0,64840.0,-64840.0,NaN,NaN,NaN,NaN,NaN,84070001,USA,NaN,NaN


In [43]:
general_state = data_state[data_state['Lat'].notnull()]

- We know that, based on the provided README file, Active cases = total confirmed - total recovered - total deaths, we can safely fill out all the NaN's in Deaths, Recorvered, and Active with 0, since they are all mutually exclusive.

In [71]:
general_state = data_state[data_state['Lat'].notnull()]

- We know that, based on the provided README file, Active cases = total confirmed - total recovered - total deaths, we can safely fill out all the NaN's in Deaths, Recorvered, and Active with 0, since they are all mutually exclusive.

In [72]:
general_state[['Recovered']] = general_state[['Recovered']].fillna(0)

In [73]:
general_state[['Active']] = general_state[['Active']].fillna(0)

In [74]:
general_state[['Mortality_Rate']] = general_state[['Mortality_Rate']].fillna(0)

- Notice that US Hospitalization Rate (%): = Total number hospitalized / Number confirmed cases, so if the number of People_Hospitalized is NaN or 0, we can logically fill in 0 for all the NaN's in these two columns

In [75]:
general_state[['People_Hospitalized']] = general_state[['People_Hospitalized']].fillna(0)

In [76]:
general_state[['Hospitalization_Rate']] = general_state[['Hospitalization_Rate']].fillna(0)

- As we can that we only have one na value left

In [70]:
general_state.isna().sum()

Province_State          0
Country_Region          0
Last_Update             1
Lat                     0
Long                    0
Confirmed               0
Deaths                  0
Recovered               0
Active                  0
FIPS                    0
Incident_Rate           0
People_Tested           0
People_Hospitalized     0
Mortality_Rate          0
UID                     0
ISO3                    0
Testing_Rate            0
Hospitalization_Rate    0
dtype: int64

In [77]:
general_state[general_state['Last_Update'].isnull()]

,Province_State,Country_Region,Last_Update,Lat,Long,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate
2,American Samoa,US,NaN,-14.271,-170.132,0,0,0.0,0.0,60.0,0.0,3.0,0.0,0.0,16,ASM,5.391708,0.0


- Since "American Samoa" has such a small number of cases, it will have relatively small effect on our prediction, so we decided to remove this area to keep the consistency of our dataframe.

In [78]:
general_state = general_state[general_state['Last_Update'].notnull()]

### 2. Join "abridged_couties" with "time_series_covid19_deaths_US"

- abridged_counties contains wonderful information about population health condition as well as population demograpics, and time_series_covid19_deaths_US has a wonderful pattern of the times series of confirms in the U.S., so it would be a lot easier to select feature from this join table rather than subseting data separately from each data set when selecting features.

In [80]:
combined_dead = data_time_de.merge(data_counties, left_on= 'Admin2', right_on = 'CountyName')

In [82]:
combined_dead.head(5)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,>500 gatherings,public schools,restaurant dine-in,entertainment/gym,federal guidelines,foreign travel ban,SVIPercentile,HPSAShortage,HPSAServedPop,HPSAUnderservedPop
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.4354,NaN,NaN,NaN
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.2162,NaN,NaN,NaN
2,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,737501.0,737502.0,737504.0,737504.0,737500.0,737495.0,0.8666,3.74,11730.0,11208.0
3,84013009,US,USA,840,13009.0,Baldwin,Georgia,US,33.068823,-83.247017,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.2162,NaN,NaN,NaN
4,84013009,US,USA,840,13009.0,Baldwin,Georgia,US,33.068823,-83.247017,...,737501.0,737502.0,737504.0,737504.0,737500.0,737495.0,0.8666,3.74,11730.0,11208.0


### 2. Join "abridged_couties" with "time_series_covid19_confirmed_US"

- abridged_counties contains wonderful information about population health condition as well as population demograpics, and time_series_covid19_confirmed_US has a wonderful pattern of the times series of confirms in the U.S., so it would be a lot easier to select feature from this join table rather than subseting data separately from each data set when selecting features.

In [83]:
combined_conf = data_time_conf.merge(data_counties, left_on= 'Admin2', right_on = 'CountyName')

In [84]:
combined_conf.head(5)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,>500 gatherings,public schools,restaurant dine-in,entertainment/gym,federal guidelines,foreign travel ban,SVIPercentile,HPSAShortage,HPSAServedPop,HPSAUnderservedPop
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.4354,NaN,NaN,NaN
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.2162,NaN,NaN,NaN
2,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,737501.0,737502.0,737504.0,737504.0,737500.0,737495.0,0.8666,3.74,11730.0,11208.0
3,84013009,US,USA,840,13009.0,Baldwin,Georgia,US,33.068823,-83.247017,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.2162,NaN,NaN,NaN
4,84013009,US,USA,840,13009.0,Baldwin,Georgia,US,33.068823,-83.247017,...,737501.0,737502.0,737504.0,737504.0,737500.0,737495.0,0.8666,3.74,11730.0,11208.0


### 3. Exporting